## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements (*Note: ocdskingfishercolab installs google-colab, which expects specific versions of pandas and numpy*):


In [ ]:
! pip install --upgrade pip > pip.log
! pip install --upgrade 'ocdskingfishercolab<0.4' ipywidgets psycopg2-binary >> pip.log

Import packages and load extensions:

In [ ]:
from collections import Counter

import numpy as np
import pandas as pd
from google.colab.data_table import DataTable
from google.colab.files import download
from ipywidgets import widgets
from ocdskingfishercolab import (
    authenticate_gspread,
    calculate_coverage,
    download_dataframe_as_csv,
    format_thousands,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
    set_spreadsheet_name,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

Set the name of the spreadsheet to export results to, when using `save_dataframe_to_sheet()`:

In [ ]:
spreadsheet_name = "feedback_results"

set_spreadsheet_name(spreadsheet_name)

Configure the notebook environment:

In [ ]:
# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()

## Charts Setup
*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

In [ ]:
! pip install --upgrade altair >> pip.log

Import chart packages and define chart functions:

In [ ]:
import altair as alt


def plot_release_count(release_counts):
    # check if input contains the right columns
    if not set(["collection_id", "release_type", "release_count", "ocid_count"]).issubset(release_counts.columns):
        raise ValueError(
            "Input data must contain the following columns: collection_id, release_type, release_count, ocid_count"
        )
    chart = (
        alt.Chart(release_counts)
        .mark_bar()
        .encode(
            x=alt.X(
                "release_count",
                type="ordinal",
                axis=alt.Axis(title="release count", labelAngle=0),
            ),
            y=alt.Y(
                "ocid_count",
                type="quantitative",
                axis=alt.Axis(title="ocid count", format="~s", tickCount=5),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                title="release type",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
            ),
            tooltip=[
                alt.Tooltip("release_count", title="release count"),
                alt.Tooltip("ocid_count", title="ocid count", format="~s"),
                alt.Tooltip("release_type", title="release type"),
                alt.Tooltip("collection_id", title="collection id"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )

    return chart


def plot_objects_per_stage(objects_per_stage):
    # check if input contains the right columns
    if not set(["stage", "object_count"]).issubset(objects_per_stage.columns):
        raise ValueError("Data must contain columns 'stage' and 'object_count'")
    # draw chart
    chart = (
        alt.Chart(objects_per_stage)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "stage",
                type="ordinal",
                scale=alt.Scale(
                    domain=[
                        "planning",
                        "tender",
                        "awards",
                        "contracts",
                        "implementation",
                    ]
                ),
                sort=["planning", "tender", "awards", "contracts", "implementation"],
                axis=alt.Axis(title="stage", labelAngle=0),
            ),
            y=alt.Y(
                "object_count",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
            ),
            tooltip=[
                alt.Tooltip("stage", title="stage"),
                alt.Tooltip("object_count", title="number of objects"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_releases_by_month(release_dates):
    # check if input contains the right columns
    if not set(["date", "collection_id", "release_type", "release_count"]).issubset(release_dates.columns):
        raise ValueError("Data must contain columns 'date', 'collection_id', 'release_type', 'release_count'")
    # check if number of rows is more than 5000
    if release_dates.shape[0] > 5000:
        alt.data_transformers.disable_max_rows()

    # draw chart
    chart = (
        alt.Chart(release_dates)
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X("date", timeUnit="yearmonth", axis=alt.Axis(title="year and month")),
            y=alt.Y(
                "release_count",
                type="quantitative",
                aggregate="sum",
                axis=alt.Axis(title="number of releases", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "release_type",
                type="nominal",
                scale=alt.Scale(range=["#D6E100", "#FB6045", "#23B2A7", "#6C75E1"]),
                legend=alt.Legend(title="release type"),
            ),
            tooltip=[
                alt.Tooltip("date", timeUnit="yearmonth", title="date"),
                alt.Tooltip("release_count", aggregate="sum", title="number of releases"),
                alt.Tooltip("release_type", title="release type"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_objects_per_year(objects_per_year):
    # check if input contains the right columns
    if not set(["year", "tenders", "awards"]).issubset(objects_per_year.columns):
        raise ValueError("Data must contain columns 'year', 'tenders' and 'awards'")
    # draw chart
    chart = (
        alt.Chart(objects_per_year)
        .transform_fold(["tenders", "awards"])
        .mark_line(strokeWidth=3)
        .encode(
            x=alt.X(
                "year",
                type="quantitative",
                axis=alt.Axis(title="year", format=".0f", tickCount=dates.shape[0]),
            ),
            y=alt.Y(
                "value",
                type="quantitative",
                axis=alt.Axis(title="number of objects", format="~s", tickCount=5),
                scale=alt.Scale(zero=False),
            ),
            color=alt.Color(
                "key",
                type="nominal",
                title="object type",
                scale=alt.Scale(domain=["tenders", "awards"], range=["#D6E100", "#FB6045"]),
            ),
            tooltip=[
                alt.Tooltip("year", title="year", type="quantitative"),
                alt.Tooltip("value", title="number of objects", type="quantitative"),
                alt.Tooltip("key", title="object type", type="nominal"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart


def plot_top_buyers(buyers):
    # check if input contains the right columns
    if not set(["name", "total_tenders"]).issubset(buyers.columns):
        raise ValueError("Data must contain columns 'name' and 'total_tenders'")
    # draw chart
    chart = (
        alt.Chart(buyers)
        .mark_bar(fill="#d6e100")
        .encode(
            x=alt.X(
                "total_tenders",
                type="quantitative",
                axis=alt.Axis(title="number of tenders", format="~s", tickCount=5),
            ),
            y=alt.Y(
                "name",
                type="ordinal",
                axis=alt.Axis(title="buyer", labelAngle=0),
                sort=alt.SortField("total_tenders", order="descending"),
            ),
            tooltip=[
                alt.Tooltip("name", title="buyer", type="nominal"),
                alt.Tooltip("total_tenders", title="number of tenders", type="quantitative"),
            ],
        )
        .properties(
            width=600,
            height=350,
            padding=50,
            title=alt.TitleParams(text="", subtitle=[""], fontSize=18),
        )
        .configure_axis(
            titleFontSize=14,
            labelFontSize=14,
            labelPadding=5,
            ticks=False,
            domain=False,
        )
        .configure_view(strokeWidth=0)
    )
    return chart

## Setup Kingfisher Process

### Connect to the database

In [ ]:
import getpass

from ocdskingfishercolab import (
    list_collections,
    list_source_ids,
    set_search_path,
)

Enter your PostgreSQL credentials and connect to the Kingfisher Process database:

In [ ]:
user = input("Username:")
password = getpass.getpass("Password:")

# Don't show connection string after execute.
%config SqlMagic.displaycon = False

connection_string = (
    "postgresql://"
    + user
    + ":"
    + password
    + "@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require"
)
%sql $connection_string

### Choose collections and schema

*Use this section to choose the collections and schema that you want to query.*

#### Set the collection(s)

Update `collection_ids` with the `id`(s) of the [Kingfisher Process collection(s)](https://kingfisher-process.readthedocs.io/en/latest/data-model.html#collections):

In [ ]:
collection_ids = (2358, 2359)

If you don't know which collections you need, run the next cell and use the **Filter** button to filter the [collection table](https://kingfisher-process.readthedocs.io/en/latest/database-structure.html#collection-table) to find the collection(s). You can use the `source_id` column to filter on the `name` of the [Kingfisher Collect spider](https://kingfisher-collect.readthedocs.io/en/latest/spiders.html) used to collect the data. Use the value(s) from the `id` column to update the previous cell.

In [ ]:
list_collections()

#### Set the schema

Update `schema_name` with the name of the [Kingfisher Summarize schema](https://kingfisher-summarize.readthedocs.io/en/latest/index.html#how-it-works).

In [ ]:
schema_name = "view_data_collection_2358_2359"
set_search_path(schema_name)

If you don't know which schema you need, run the next cell and use the **Filter** button to filter the [selected collections table](https://kingfisher-summarize.readthedocs.io/en/latest/database.html#summaries-selected-collections) to find the schema. You can use the `collection_id` column to filter on the `id` of the collections that you identified in the previous step. Alternatively, you can filter on the `source_id` column. Use the value from the `schema` column to update the previous cell.

In [ ]:
%%sql

SELECT
    summaries.selected_collections.*,
    source_id
FROM
    summaries.selected_collections
    JOIN collection ON summaries.selected_collections.collection_id = collection.id


If you can't find a schema containing the collections that you want to query, you can create a schema using [Kingfisher Summarize](https://ocdsdeploy.readthedocs.io/en/latest/use/kingfisher-summarize.html).

## Check for data collection and processing errors

Kingfisher Collect and Kingfisher Process log messages that might indicate OCDS implementation errors or API stability issues.

Confirm any errors or warnings by manually checking the OCDS publication.

### Kingfisher Collect

See [how to review the Kingfisher Collect crawl’s log file](https://kingfisher-collect.readthedocs.io/en/latest/logs.html).

**Note:** There is an open [pull request](https://github.com/open-contracting/notebooks-ocds/pull/44) to automate this. If interested, please comment on the pull request to prioritize it.

### Kingfisher Process

Users add a note when starting a crawl and when running the `load` command.

Kingfisher Process adds notes (the `note` column) at different levels (the `code` column):

- `INFO`
  - *load*: The Kingfisher Collect crawl's reason for closing and statistics (the `data` column).
- `WARNING`
  - *compile*: When a record has undated releases, linked releases or no releases, but a compiled release can be calculated (by merging remaining dated releases, using the `compiledRelease` field, or using an undated release with a 'compiled' tag).
- `ERROR`
  - *load*: When the input data is invalid JSON.
  - *load*: When Kingfisher Collect yielded a `FileError` item.
  - *compile*: When no compiled release can be calculated for a record.

**Note:** `FileError` items in Kingfisher Process are [scheduled for removal](https://github.com/open-contracting/kingfisher-process/issues/366), as they also appear in the crawl's log file.

List the notes for each of your collections:

In [ ]:
%%sql

SELECT
    collection_id,
    code,
    note,
    data
FROM
    collection_note
WHERE
    collection_id IN :collection_ids


## Check scope



Use this section to check:

* how many releases, records and compiled releases your data contains
* what stages of the contracting process your data covers
* what date range your data covers

If you are preparing an [Ad-hoc structure and format feedback](https://docs.google.com/document/d/1_k7eA2rI-k5EH8VESkVAB73wa_qrpplL-7dKgMLTGZc/edit#heading=h.i7tpu8c49dcv), you might skip this section.

### Release and record counts

Collections in Kingfisher Process contain either [releases](https://standard.open-contracting.org/latest/en/schema/reference/), [records](https://standard.open-contracting.org/latest/en/schema/records_reference/) or [compiled releases](https://standard.open-contracting.org/latest/en/schema/records_reference/#compiled-release). Kingfisher Process creates compiled release collections from release or record collections.

Use this section to check that the data contains the expected number of releases, records and compiled releases. Where possible, you should check these numbers against the total number of results available in the frontend of the data source.

Count the number of releases, records and compiled releases, for each collection.

**Note:** These columns are not yet populated in version 2 of Kingfisher Process. Comment on [this issue](https://github.com/open-contracting/kingfisher-process/issues/370) to prioritize it.

In [ ]:
%%sql

SELECT
    id AS collection_id,
    cached_releases_count AS releases_count,
    cached_records_count AS records_count,
    cached_compiled_releases_count AS compiled_releases_count
FROM
    collection
WHERE
    id IN :collection_ids


### Contracting process stages

Use this section to check that the data covers the expected stages of the contracting process.

#### Release tags

[Release tags](https://standard.open-contracting.org/latest/en/schema/codelists/#release-tag) indicate the stage of a contracting process to which a release is related.

Count the number of releases, for each release tag:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    tag,
    count(*)
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    tag
ORDER BY
    collection_id


#### Objects per stage

In OCDS, data is organized into objects, for each stage of a contracting process. Each compiled release has: at most one `Planning` object, at most one `Tender` object, any number of `Award` objects, and any number of `Contract` objects. Each `Contract` object has at most one `Implementation` object. As such, the number of `Award` objects can exceed the number of unique OCIDs, but the number of `Tender` objects can't.

Plot a count of objects per stage:

In [ ]:
%%sql objects_per_stage <<

SELECT
    CASE WHEN paths.path = 'contracts/implementation' THEN
        'implementation'
    ELSE
        paths.path
    END AS stage,
    CASE WHEN paths.path IN ('planning', 'tender', 'contracts/implementation') THEN
        GREATEST (object_property, 0)
    ELSE
        GREATEST (array_count, 0)
    END AS object_count
FROM (
    SELECT
        unnest(ARRAY['planning', 'tender', 'awards', 'contracts', 'contracts/implementation']) AS path) AS paths
    LEFT JOIN (
        SELECT
            *
        FROM
            field_counts
        WHERE
            collection_id IN :collection_ids
            AND release_type = 'compiled_release'
            AND path IN ('planning', 'tender', 'awards', 'contracts', 'contracts/implementation')) AS field_counts USING (path)


In [ ]:
plot_objects_per_stage(objects_per_stage)

### Date ranges


Use this section to check that the data covers the expected date range.

Calculate the earliest and latest `date`, `awards/date` and `contracts/dateSigned`:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    'release_date' AS date_type,
    min(date) AS min,
    max(date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'award_date' AS date_type,
    min(first_award_date) AS min,
    max(last_award_date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'contract_datesigned' AS date_type,
    min(first_contract_datesigned) AS min,
    max(last_contract_datesigned) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type
ORDER BY
    collection_id,
    release_type,
    date_type;



### Release date distribution

Use this section to check that releases are distributed as expected.

Plot the count of releases per month:

In [ ]:
%%sql release_dates <<

SELECT
    collection_id::text,
    release_type,
    date,
    count(*) AS release_count
FROM
    release_summary rs
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id,
    release_type,
    date
ORDER BY
    date ASC;



In [ ]:
# Resample by month
release_dates = release_dates.set_index("date")
release_dates = release_dates.groupby(["collection_id", "release_type"]).resample("M").sum()

plot_releases_by_month(release_dates)

### Extensions

Use this section to check which extensions the data uses.

List the extensions declared in the package metadata:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    jsonb_array_elements(package_data -> 'extensions') AS ocds_extension,
    count(*) AS count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND package_data IS NOT NULL
GROUP BY
    collection_id,
    release_type,
    ocds_extension
ORDER BY
    collection_id,
    release_type,
    count DESC;

